# pandas入门

+ 具备按轴自动或显式数据对齐功能的数据结构。这可以防止许多由于数据未对齐以及来自不同数据源（索引方式不同）的数据而导致的常见错误。
+ 集成时间序列功能。
+ 既能处理时间序列数据也能处理非时间序列数据的数据结构。
+ 数学运算和约简（比如对某个轴求和）可以根据不同的元数据（轴编号）执行。
+ 灵活处理缺失数据。
+ 合并及其他出现在常见数据库（例如基于SQL的）中的关系型运算。

## 目录
+ pandas的数据结构介绍
 + Series
   + 创建Series
   + 修改Series
   + 修改Series的索引
   + Series的属性
 + DataFrame
   + 创建DataFrame
   + 修改DataFrame的列
   + 修改DataFrame的行标、列标
   + DataFrame的属性
 + 索引对象
   + pandas中主要的Index对象
   + Index的方法和属性
 + in关键字

 
+ 基本功能
 + 索引、选取和过滤
   + Series
   + DataFrame
     + obj[val]
     + ix方法
     + reindex方法
     + xs方法
     + icol、irow方法
     + get_value、set_value方法
 + 丢弃指定轴上的项
 + 算术运算
   + 数据对齐
   + 在算术方法中填充值
   + DataFrame和Series之间的运算
 + 函数应用和映射
   + 对pandas对象直接使用
   + apply方法族
     + apply方法
     + applymap方法
     + map方法
 + 排序
   + 按索引排序
   + 按值排序
 + 排名
   + 用于破坏平级关系的method选项
 + 带有重复值的轴索引


+ 汇总和计算描述统计
 + 约简方法的选项
 + 描述和汇总统计
 + 相关系数与协方差
 + 唯一值、值计数以及成员资格
   + Series
   + DataFrame


+ 处理缺失数据
 + 判断是否有缺失值
 + 滤除缺失数据
 + 填充缺失数据


+ 层次化索引
 + Series
 + DataFrame
 + 创建并使用MultiIndex
 + 重排分级顺序
   + 分层索引互换
   + 分层索引排序
 + 根据级别汇总统计
 + 使用DataFrame的列
   + 列转换为行索引
   + 行索引转换为列


+ 其他有关pandas的话题
 + 整数索引
   + 在pandas中尽量避免使用整数索引，而使用标签索引
   + 获取可靠的、不考虑索引类型的、基于位置的索引
 + 面板数据
   + 面板数据的索引
   + 面板数据与“堆积式”DataFrame（层次化索引）的转换

In [1]:
from numpy.random import randn
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

## pandas的数据结构介绍

### Series
+ Series可以被视为一维数组
+ Series可以被视为**定长的有序**字典

#### 创建Series

+ 一维数组

In [2]:
obj = Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [3]:
obj2 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c']) # index参数，设置索引
obj2

d    4
b    7
a   -5
c    3
dtype: int64

+ 字典，无index

In [4]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = Series(sdata) # Series中的索引就是原字典的键
obj3

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

+ 字典，有index

In [5]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = Series(sdata, index=states) # 自动匹配，原字典中无对应项，则赋值NaN
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

#### 修改Series
+ 通过索引方式返回的列只是相应数据的视图，因此对返回的Series所做的任何修改会反映到源Series上
+ 通过Series的copy方法可显式地复制列

In [6]:
obj[1:2] = 2
obj

0    4
1    2
2   -5
3    3
dtype: int64

#### 修改Series的索引

In [7]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan'] # 索引可以通过赋值的方式就地修改
obj

Bob      4
Steve    2
Jeff    -5
Ryan     3
dtype: int64

#### Series的属性

In [8]:
obj.values # 返回值

array([ 4,  2, -5,  3], dtype=int64)

In [9]:
obj.index # 返回索引

Index([u'Bob', u'Steve', u'Jeff', u'Ryan'], dtype='object')

In [10]:
obj4.name = 'population' # Series对象本身有name属性
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [11]:
obj4.index.name = 'state' # Series的索引有name属性
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

### DataFrame
+ DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔值等）
+ DataFrame既有行索引也有列索引，它可以被看做有Series组成的字典（共用同一个索引）

#### 创建DataFrame

+ 二维ndarray  
数据矩阵，还可以通过index、columns参数设置行标和列标


+  由列表或元组组成的列表  
类似于“二维ndarray”


+ NumPy的MaskedArray  
类似于“二维ndarray”的情况，只是掩码值在结果DataFrame会变成NA/缺失值

In [12]:
data = np.arange(4).reshape(2, 2)
frame = DataFrame(data, index = ['a', 'b'], columns = ['one', 'two'])
frame

,one,two
a,0,1
b,2,3


+ 由数组、列表或元组组成的字典  
 + 每个序列会变成DataFrame中的一列。所有序列的长度必须相同
 + 通过index参数设置行索引
 + 列名是字典的键，通过columns参数设置选取的列及排列顺序，如果传入的列在原字典中找不到，就会被赋值NaN
 
 
+ NumPy的结构化/记录数组  
类似于“由数组组成的字典”

In [13]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}

In [14]:
frame2 = DataFrame(data, columns=['year', 'state', 'pop', 'debt'], # 'debt'列在原字典中找不到，被赋值NaN
                   index=['one', 'two', 'three', 'four', 'five'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


+ 由Series组成的字典  
每个Series会成为一列。如果没有显式指定索引，则各Series的索引会被合并成结果的行索引


+ 由字典组成的字典  
各内层字典会成为一列。键会被合并成结果的行索引，跟“由Series组成的字典”的情况一样

In [15]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

In [16]:
frame3 = DataFrame(pop) # 内层字典的键会被合并、排序形成结果的行索引
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [17]:
DataFrame(pop, index=[2001, 2002, 2003]) # 显式指定索引，原字典中没有键的赋值NaN

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


In [18]:
pdata = {'Ohio': frame3['Ohio'][:-1],
         'Nevada': frame3['Nevada'][:2]} # Series可以被视为字典，两种情况用法相同
DataFrame(pdata)

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7


+ 字典或Series的列表  
各项将会成为DataFrame的一**行**。字典键或Series索引的并集将会成为DataFrame的列标

+ 另一个DataFrame  
该DataFrame的索引将会被沿用，除非显式指定了其他索引

#### 修改DataFrame的列
+ 通过索引方式返回的列只是相应数据的视图，因此对返回的Series所做的任何修改会反映到源DataFrame上
+ 通过Series的copy方法可显式地复制列

+ 通过标量赋值

In [19]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5


+ 通过数组、列表等序列赋值  
长度必须跟DataFrame的长度相匹配

In [20]:
frame2['debt'] = np.arange(5.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0


+ 通过Series赋值  
精确匹配DataFrame的索引，所有的空位会被填上缺失值

In [21]:
val = Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7


+ 为不存在的列赋值  
创建出一个新列

In [22]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False


+ 删除列

In [23]:
del frame2['eastern'] # 关键字del用于删除列
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7


#### 修改DataFrame的行标、列标

In [24]:
frame.index = ['one', 'two']
frame.columns = ['a', 'b']
frame

,a,b
one,0,1
two,2,3


#### DataFrame的属性

In [25]:
frame2.index # 返回DataFrame的行名

Index([u'one', u'two', u'three', u'four', u'five'], dtype='object')

In [26]:
frame2.columns # 返回DataFrame的列名

Index([u'year', u'state', u'pop', u'debt'], dtype='object')

In [27]:
frame2.year # 选取列，等同于frame2['year']，列名即是产生的Series的对象name

one      2000
two      2001
three    2002
four     2001
five     2002
Name: year, dtype: int64

In [28]:
frame3.T # 对DataFrame进行转置

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


In [29]:
frame3.index.name = 'year' # 设置index的name属性
frame3.columns.name = 'state' # 设置columns的name属性
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [30]:
frame3.values # values属性会以二维ndarray的形式返回DataFrame中的数据

array([[ nan,  1.5],
       [ 2.4,  1.7],
       [ 2.9,  3.6]])

In [31]:
frame2.values # DataFrame各列数据类型不同时，值数组的数据类型会选用能兼容所有列的数据类型

array([[2000L, 'Ohio', 1.5, nan],
       [2001L, 'Ohio', 1.7, -1.2],
       [2002L, 'Ohio', 3.6, nan],
       [2001L, 'Nevada', 2.4, -1.5],
       [2002L, 'Nevada', 2.9, -1.7]], dtype=object)

### 索引对象
+ pandas的索引对象负责管理轴标签和其他元数据（如轴名称等）
+ 构建Series或DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个Index
+ Index对象是不可修改的，这样才能使Index对象在多个数据结构之间安全共享

#### pandas中主要的Index对象
 + Index  
最泛化的Index对象，将轴标签表示为一个由Python对象组成的NumPy数组
 + Int64Index  
针对整数的特殊Index
 + MultiIndex  
“层次化”索引对象，表示单个轴上的多层索引。可以看做由元组组成的数组
 + DatetimeIndex  
存储纳秒级时间戳（用NumPy的datetime64类型表示
 + PeriodIndex  
针对Period数据（时间间隔）的特殊Index

#### Index的方法和属性
 + append  
 连接另一个Index对象，产生一个新的Index
 + diff  
 计算差集，并得到一个Index
 + intersection  
 计算交集
 + union  
 计算并集
 + isin  
 计算一个指示各值是否都包含在参数集合中的布尔型数组
 + delete  
 删除索引i处的元素，并得到新的Index
 + drop  
 删除传入的值，并得到新的Index
 + insert  
 将元素插入到索引i处，并得到新的Index
 + is_monotonic  
 当各元素均大于等于前一个元素时，返回True
 + is_unique  
 当Index没有重复值时，返回True
 + unique  
 计算Index中唯一值的数组

In [32]:
obj = Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index([u'a', u'b', u'c'], dtype='object')

In [33]:
obj.index = [1, 2, 3] #Index可以用这种方式“整体”修改
obj.index

Int64Index([1, 2, 3], dtype='int64')

In [34]:
index = pd.Index(np.arange(3))
obj2 = Series([1.5, -2.5, 0], index=index) # 事先构建Index对象，赋值给新建的Series和DataFrame
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [35]:
obj2.index is index # Index对象的不可修改性，可以在多个数据结构之间安全共享

True

### in关键字

#### Series

In [36]:
'b' in obj2 # Series可被视为字典，in关键字用法同字典

False

In [37]:
'b' in obj2.index

False

#### DataFrame

In [38]:
'Ohio' in frame3 # 在DataFrame中使用in关键字，默认在列名中搜寻

True

In [39]:
'Ohio' in frame3.columns

True

In [40]:
2003 in frame3.index

False

## 基本功能

### 索引、选取和过滤

#### Series

In [41]:
obj = Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [42]:
obj['b'] # 标签索引

1.0

In [43]:
obj[['b', 'a', 'd']] # 标签数组索引

b    1.0
a    0.0
d    3.0
dtype: float64

In [44]:
obj['b':'c'] # 标签切片索引，同普通Python切片不同，标签切片包含末端

b    1.0
c    2.0
dtype: float64

In [45]:
obj[1] # 整数索引

1.0

In [46]:
obj[2:4] # 切片索引

c    2.0
d    3.0
dtype: float64

In [47]:
obj[obj < 2] # 布尔型索引

a    0.0
b    1.0
dtype: float64

In [48]:
obj[[1, 3]] # （花式索引）整数数组索引

b    1.0
d    3.0
dtype: float64

#### DataFrame

In [49]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


##### obj[val]
 + 通过标签、标签数组、整数数组索引选取**列**
 + 通过切片、布尔型索引选取**行**
 + 通过布尔型DataFrame进行索引

In [50]:
data['two'] # 标签索引

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [51]:
data[['three', 'one']] # 标签数组索引

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [52]:
data[[2, 0]] # 整数数组索引，根据列所在位置，同标签数组索引

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [53]:
data[:2] # 切片索引，按行索引

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [54]:
data[data['three'] > 5] # 布尔型索引，按行索引

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [55]:
data[data < 5] = 0 # 布尔型DataFrame索引，按整个数据框索引
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


##### ix方法  
使用索引字段ix的情况下，**用于Series的7种索引方式**都可以使用
 + obj.ix[val]  
选取单个行或一组行
 + obj.ix[:,val]  
选取单个列或列子集
 + obj.ix[val1, val2]  
同时选取行和列

In [56]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [57]:
data.ix['Colorado', ['two', 'three']] # 降维

two      5
three    6
Name: Colorado, dtype: int32

In [58]:
data.ix[['Colorado', 'Utah'], [3, 0, 1]] # 不降维

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [59]:
data.ix[[2]] # 按行索引，不降维

,one,two,three,four
Utah,8,9,10,11


In [60]:
data.ix[2] # 按行索引，降维

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [61]:
data.ix[:'Utah', 'two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [62]:
data.ix[data.three > 5, :3]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


##### reindex方法
+ reindex函数的参数
 + index  
 用作索引的新序列。既可以是Index实例，也可以是其他序列型的Python数据结构。Index会被完全使用，就像没有任何复制一样
 + method  
 插值（填充）方式
 + fill_value  
 在重新索引的过程中，需要引入缺失值时使用的替代值
 + limit  
 前向或后向填充时的最大填充量
 + level  
 在MultiIndex的指定级别上匹配简单索引，否则选取其子集
 + copy  
 默认为True，无论如何都复制；如果为False，则新旧相等就不复制
 
 
+ 插值的method选项
 + ffill或pad  
 前向填充（或搬运）值
 + bfill或backfill  
 后向填充（或搬运）值
 + nearest  
 搬运索引距离最近的值

###### Series

In [63]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [64]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e']) # 如果某个索引值不存在，引入缺失值
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [65]:
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0) # fill_value填充缺失值

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

In [66]:
obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [67]:
obj3.reindex(range(6), method='ffill') # 插值

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

###### DataFrame

In [68]:
frame = DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                  columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [69]:
frame2 = frame.reindex(['a', 'b', 'c', 'd']) # 如果仅传入一个序列，默认重新索引行
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [70]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states) # 使用columns参数，重新索引列

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [71]:
frame.reindex(index=['a', 'b', 'c', 'd'], method='ffill',
              columns=states) # 同时重新索引行、列，插值只能按行应用

,Texas,Utah,California
a,1,NaN,2
b,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [72]:
frame.ix[['a', 'b', 'c', 'd'], states] # ix方法也可以重新索引，但没有reindex方法那样多的参数

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


##### xs方法
根据标签选取单行或单列，并返回一个Series

##### icol、irow方法
根据整数位置选取单列或单行，并返回一个Series

##### get_value、set_value方法
根据行标签和列标签选取或设置单个值

### 丢弃指定轴上的项
drop方法返回一个在指定轴上删除了指定值的新对象

#### Series

In [73]:
obj = Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [74]:
obj.drop('d')

a    0.0
b    1.0
c    2.0
e    4.0
dtype: float64

In [75]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

#### DataFrame

In [76]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [77]:
data.drop(['Colorado', 'Ohio']) # 默认按行丢弃

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [78]:
data.drop('two', axis=1) # 按列丢弃

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [79]:
data.drop(['two', 'four'], axis=1) # 按列丢弃

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


### 算术运算

#### 数据对齐
在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集

In [80]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [81]:
s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [82]:
s1 + s2 # 数据对齐操作在不重叠的索引处引入了NA值

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [83]:
df1 = DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [84]:
df2 = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [85]:
df1 + df2 # DataFrame相加，索引和列为原来2个DataFrame的并集，且只有2个DataFrame的交集位置才有值，其他位置都是NaN

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


#### 在算术方法中填充值

In [86]:
df1 = DataFrame(np.arange(12.).reshape((3, 4)), columns=list('abcd'))
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [87]:
df2 = DataFrame(np.arange(20.).reshape((4, 5)), columns=list('abcde'))
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [88]:
df1.add(df2, fill_value=0) # 通过fill_value参数填充缺失值

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


##### 算术方法
+ add  
用于加法（+）的方法
+ sub  
用于减法（-）的方法
+ div  
用于除法（/）的方法
+ mul  
用于乘法（*）的方法

#### DataFrame和Series之间的运算

##### 匹配列索引，沿着行向下广播
中缀符

In [89]:
frame = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [90]:
series = frame.ix[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [91]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [92]:
series2 = Series(range(3), index=['b', 'e', 'f'])
series2

b    0
e    1
f    2
dtype: int64

In [93]:
frame + series2 # 如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象会被重新索引以形成并集，缺失索引赋值NaN

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


##### 匹配行索引，沿着列向右广播
算术方法

In [94]:
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [95]:
frame.sub(series3, axis=0) # 传入的轴号就是索引匹配的轴

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


### 函数应用和映射
NumPy的ufuncs（元素级数组方法）也可以用于操作pandas对象

In [96]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,2.074320,0.988713,0.870912
Ohio,-0.376540,-0.432501,0.144023
Texas,1.493258,1.257546,0.140596
Oregon,1.327552,-0.626324,0.323546


#### 对pandas对象直接使用

In [97]:
np.abs(frame)

,b,d,e
Utah,2.074320,0.988713,0.870912
Ohio,0.376540,0.432501,0.144023
Texas,1.493258,1.257546,0.140596
Oregon,1.327552,0.626324,0.323546


#### apply方法族

##### apply方法
将函数应用到由DataFrame各列或行所形成的一维数组上

###### 函数返回的是标量值，最终结果是Series

In [98]:
f = lambda x: x.max() - x.min() # 自定义求极差函数

In [99]:
frame.apply(f) # 默认对列应用函数

b    2.450859
d    1.883869
e    0.730315
dtype: float64

In [100]:
frame.apply(f, axis=1) # 0为列，1为行

Utah      1.203408
Ohio      0.576525
Texas     1.352661
Oregon    1.953875
dtype: float64

###### 函数返回的是由多个值组成的Series，最终结果是DataFrame

In [101]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,-0.37654,-0.626324,0.140596
max,2.07432,1.257546,0.870912


##### applymap方法
将函数应用到DataFrame的元素级

In [102]:
format = lambda x: '%.2f' % x # 返回浮点值的格式化字符串
frame.applymap(format)

,b,d,e
Utah,2.07,0.99,0.87
Ohio,-0.38,-0.43,0.14
Texas,1.49,1.26,0.14
Oregon,1.33,-0.63,0.32


##### map方法
将函数应用到Series的元素级

In [103]:
frame['e'].map(format)

Utah      0.87
Ohio      0.14
Texas     0.14
Oregon    0.32
Name: e, dtype: object

### 排序

#### 按索引排序

##### Series

In [104]:
obj = Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [105]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

##### DataFrame

In [106]:
frame = DataFrame(np.arange(8).reshape((2, 4)), index=['three', 'one'],
                  columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [107]:
frame.sort_index() # 默认按行索引排序

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [108]:
frame.sort_index(axis=1) # 按列索引排序

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [109]:
frame.sort_index(axis=1, ascending=False) # 按降序排序

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


#### 按值排序

##### Series

In [110]:
obj = Series([4, np.nan, 7, np.nan, -3, 2])
obj

0    4.0
1    NaN
2    7.0
3    NaN
4   -3.0
5    2.0
dtype: float64

In [111]:
obj.sort_values() # 缺失值默认放到末尾

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

##### DataFrame

In [112]:
frame = DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,a,b
0,0,4
1,1,7
2,0,-3
3,1,2


In [113]:
frame.sort_values(by=['a', 'b']) # 根据一个或多个列中的值进行排序

,a,b
2,0,-3
0,0,4
3,1,2
1,1,7


### 排名

#### 用于破坏平级关系的method选项
+ 'average'  
默认：在相等分组中，为各个值分配平均排名
+ 'min'  
使用整个分组的最小排名
+ 'max'  
使用整个分组的最大排名
+ 'first'  
按值在原始数据中的出现顺序分配排名

#### Series

In [114]:
obj = Series([7, -5, 7, 4, 2, 0, 4])
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

In [115]:
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [116]:
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

#### DataFrame

In [117]:
frame = DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                   'c': [-2, 5, 8, -2.5]})
frame

,a,b,c
0,0,4.3,-2.0
1,1,7.0,5.0
2,0,-3.0,8.0
3,1,2.0,-2.5


In [118]:
frame.rank() # 默认在列上排名

,a,b,c
0,1.5,3.0,2.0
1,3.5,4.0,3.0
2,1.5,1.0,4.0
3,3.5,2.0,1.0


In [119]:
frame.rank(axis=1) # 在行上排名

,a,b,c
0,2.0,3.0,1.0
1,1.0,3.0,2.0
2,2.0,1.0,3.0
3,2.0,3.0,1.0


### 带有重复值的轴索引
pandas对象不要求轴索引唯一

In [120]:
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

In [121]:
obj.index.is_unique # 判断轴索引是否唯一

True

#### Series

In [122]:
obj = Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

In [123]:
obj['a'] # 索引对应多个值，返回一个Series

a    0
a    1
dtype: int64

#### DataFrame

In [124]:
df = DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,-0.370505,1.445988,2.008473
a,-1.481345,-0.850029,0.825789
b,2.357963,-0.576192,-0.741420
b,0.082217,-1.112073,-0.109017


In [125]:
df.ix['b'] # 索引对应多个值，返回一个DataFrame

,0,1,2
b,2.357963,-0.576192,-0.741420
b,0.082217,-1.112073,-0.109017


## 汇总和计算描述统计
从Series中提取单个值，或从DataFrame的行或列中提取一个Series

### 约简方法的选项
+ axis  
约简的轴。DataFrame的行用0， 列用1
+ skipna  
排除缺失值，默认值为True
+ level  
如果轴是层次化索引的（即MultiIndex），则根据level分组约简

### 描述和汇总统计
+ count  
非NA值的数量
+ describe  
针对Series或各DataFrame列计算汇总统计
+ min、max  
计算最小值和最大值
+ argmin、argmax  
计算能够获取到最小值和最大值的索引位置（整数）
+ idxmin、idxmax  
计算能够获取到最小值和最大值的索引值
+ quantile  
计算样本的分位数（0到1）
+ sum  
值的总和
+ mean  
值的平均数
+ median  
值的算术中位数（50%分位数）
+ mad  
根据平均值计算平均绝对离差
+ var  
样本值的方差
+ std  
样本值的标准差
+ skew  
样本值的偏度（三阶矩）
+ kurt  
样本值的峰度（四阶矩）
+ cumsum  
样本值的累计和
+ cummin、cummax  
样本值的累计最大值和累计最小值
+ cumprod  
样本值的累计积
+ diff  
计算一阶差分（对时间序列很有用）
+ pct_change  
计算百分数变化

In [126]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5], # np.nan产生NaN值
                [np.nan, np.nan], [0.75, -1.3]],
               index=['a', 'b', 'c', 'd'],
               columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [127]:
df.sum() # 默认按列求和

one    9.25
two   -5.80
dtype: float64

In [128]:
df.sum(axis=1) # 按行求和

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [129]:
df.mean(axis=1, skipna=False) # 默认跳过NA值，可以选择禁用该功能

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

In [130]:
df.describe() # 数值型pandas对象的汇总统计，DataFrame默认按列汇总统计

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [131]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

In [132]:
obj.describe() # 非数值型pandas对象的汇总统计

count     16
unique     3
top        a
freq       8
dtype: object

### 相关系数与协方差

In [133]:
import pandas_datareader.data as web # 需要安装pandas-datareader包

all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']:
    all_data[ticker] = web.get_data_yahoo(ticker)

price = DataFrame({tic: data['Adj Close']
                   for tic, data in all_data.iteritems()})
volume = DataFrame({tic: data['Volume']
                    for tic, data in all_data.iteritems()})

In [134]:
price.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571


In [135]:
volume.head()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400


In [136]:
returns = price.pct_change()
returns.tail() # 按列计算价格的百分数变化

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-06,0.007430,0.000502,-0.001273,0.001735
2016-10-07,0.001493,-0.002291,-0.007713,0.001039
2016-10-10,0.017447,0.014011,0.008672,0.004152
2016-10-11,0.002154,-0.003652,-0.014202,-0.014645
2016-10-12,0.008942,0.003920,-0.003230,-0.001399


#### Series
返回标量值

In [137]:
returns.MSFT.corr(returns.IBM) # 计算两个Series中重叠的、非NA的、按索引对齐的值的相关系数

0.50425379801817982

In [138]:
returns.MSFT.cov(returns.IBM) # 计算协方差

8.9474435098775738e-05

#### DataFrame
返回矩阵

In [139]:
returns.corr() # 以DataFrame的形式返回相关系数矩阵

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.408275,0.387196,0.391071
GOOG,0.408275,1.000000,0.407553,0.466734
IBM,0.387196,0.407553,1.000000,0.504254
MSFT,0.391071,0.466734,0.504254,1.000000


In [140]:
returns.cov() # 返回协方差矩阵

,AAPL,GOOG,IBM,MSFT
AAPL,0.000278,0.000108,0.000078,0.000096
GOOG,0.000108,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000147,0.000089
MSFT,0.000096,0.000108,0.000089,0.000215


###### DataFrame的corrwith方法

In [141]:
returns.corrwith(returns.IBM) # 传入Series，计算DataFrame的各列与传入的Series的相关系数

AAPL    0.387196
GOOG    0.407553
IBM     1.000000
MSFT    0.504254
dtype: float64

In [142]:
returns.corrwith(volume) # 传入DataFrame，计算按列名配对的相关系数；传入axis = 1可按行进行计算

AAPL   -0.076353
GOOG   -0.006668
IBM    -0.201842
MSFT   -0.095449
dtype: float64

### 唯一值、值计数以及成员资格

#### Series

In [143]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

##### unique方法
计算Series中的唯一值数组，按发行的顺序返回

In [144]:
uniques = obj.unique() # 得到Series中的唯一值数组
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

##### value_counts方法、pd.value_counts函数
返回一个Series，其索引为唯一值，其值为频率，按计数值降序排列

In [145]:
obj.value_counts() # 计算Series中各值出现的频率，并默认按降序排列

c    3
a    3
b    2
d    1
dtype: int64

In [146]:
pd.value_counts(obj.values, sort=False) # vale_counts也是pandas顶级函数

a    3
c    3
b    2
d    1
dtype: int64

##### isin方法
计算一个表示“Series各值是否包含于传入的值序列中”的布尔型数组，可用于选取Series或DataFrame列（也就是Series）中数据的子集

In [147]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [148]:
obj[mask] # 选取子集

0    c
5    b
6    b
7    c
8    c
dtype: object

####  DataFrame
与apply方法结合，用于DataFrame

In [149]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
                  'Qu2': [2, 3, 1, 2, 3],
                  'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [150]:
result = data.apply(pd.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


## 处理缺失数据

### 判断是否有缺失值
+ isnull  
返回一个含有布尔值的对象，这些布尔值表示哪些值是缺失值NA，该对象的类型和源数据一样
+ notnull  
isnull的否定式

In [151]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado']) # np.nan创建缺失值
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [152]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [153]:
string_data.notnull()

0     True
1     True
2    False
3     True
dtype: bool

In [154]:
string_data[0] = None
string_data.isnull() # None值也会被当做NA处理

0     True
1    False
2     True
3    False
dtype: bool

### 滤除缺失数据
+ dropna  
根据各标签的值中是否存在缺失数据对轴标签进行过滤，可通过阈值**thresh参数**调节对缺失值的容忍度

#### Series

In [155]:
from numpy import nan as NA
data = Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [156]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [157]:
data[data.notnull()] # 通过布尔型索引也可以达到dropna的效果

0    1.0
2    3.5
4    7.0
dtype: float64

#### DataFrame

In [158]:
data = DataFrame([[1., 6.5, 3.], [1., NA, NA],
                  [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [159]:
cleaned = data.dropna() # dropna默认丢弃任何含有缺失值的行
cleaned

,0,1,2
0,1.0,6.5,3.0


In [160]:
data.dropna(how='all') # 丢弃全为NA的行

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [161]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [162]:
data.dropna(axis=1, how='all') # 丢弃全为NA的列

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [163]:
df = DataFrame(np.random.randn(7, 3))
df.ix[:4, 1] = NA; df.ix[:2, 2] = NA
df

,0,1,2
0,-1.019591,NaN,NaN
1,-0.326863,NaN,NaN
2,1.242588,NaN,NaN
3,-0.123944,NaN,-1.149112
4,0.064778,NaN,-0.104764
5,-0.124781,0.410750,-0.288375
6,-0.218836,0.206677,-0.605199


In [164]:
df.dropna(thresh=2) # thresh参数值的含义：每行/列中至少有多少个非NA值才保留

,0,1,2
3,-0.123944,NaN,-1.149112
4,0.064778,NaN,-0.104764
5,-0.124781,0.410750,-0.288375
6,-0.218836,0.206677,-0.605199


### 填充缺失数据
+ fillna  
用指定值或插值方法（如ffill或bfill）填充缺失数据

#### fillna方法的参数
+ value  
用于填充缺失值的标量值或字典对象
+ method  
插值方式。如果函数调用时未指定其他参数的话，默认为“ffill”
+ axis  
待填充的轴，默认axis = 0
+ inplace  
修改调用者对象而不产生副本
+ limit  
（对于前向和后向填充）可以连续填充的最大数量

In [165]:
df

,0,1,2
0,-1.019591,NaN,NaN
1,-0.326863,NaN,NaN
2,1.242588,NaN,NaN
3,-0.123944,NaN,-1.149112
4,0.064778,NaN,-0.104764
5,-0.124781,0.410750,-0.288375
6,-0.218836,0.206677,-0.605199


In [166]:
df.fillna(0) # 传入标量值，将缺失值替换为该常数值

,0,1,2
0,-1.019591,0.000000,0.000000
1,-0.326863,0.000000,0.000000
2,1.242588,0.000000,0.000000
3,-0.123944,0.000000,-1.149112
4,0.064778,0.000000,-0.104764
5,-0.124781,0.410750,-0.288375
6,-0.218836,0.206677,-0.605199


In [167]:
df.fillna({1: 0.5, 2: -1}) # 传入字典，对不同的列填充不同的值

,0,1,2
0,-1.019591,0.500000,-1.000000
1,-0.326863,0.500000,-1.000000
2,1.242588,0.500000,-1.000000
3,-0.123944,0.500000,-1.149112
4,0.064778,0.500000,-0.104764
5,-0.124781,0.410750,-0.288375
6,-0.218836,0.206677,-0.605199


In [168]:
_ = df.fillna(0, inplace=True) # 默认返回新对象，通过inplace参数设置就地修改
df

,0,1,2
0,-1.019591,0.000000,0.000000
1,-0.326863,0.000000,0.000000
2,1.242588,0.000000,0.000000
3,-0.123944,0.000000,-1.149112
4,0.064778,0.000000,-0.104764
5,-0.124781,0.410750,-0.288375
6,-0.218836,0.206677,-0.605199


In [169]:
df = DataFrame(np.random.randn(6, 3))
df.ix[2:, 1] = NA; df.ix[4:, 2] = NA
df

,0,1,2
0,-0.005489,-0.786529,-0.161098
1,-1.240781,-1.401992,-0.453874
2,0.063155,NaN,-1.314886
3,-1.365589,NaN,0.531838
4,0.398513,NaN,NaN
5,-0.345927,NaN,NaN


In [170]:
df.fillna(method='ffill') # 插值

,0,1,2
0,-0.005489,-0.786529,-0.161098
1,-1.240781,-1.401992,-0.453874
2,0.063155,-1.401992,-1.314886
3,-1.365589,-1.401992,0.531838
4,0.398513,-1.401992,0.531838
5,-0.345927,-1.401992,0.531838


In [171]:
df.fillna(method='ffill', limit=2) # 设置连续填充的最大数量

,0,1,2
0,-0.005489,-0.786529,-0.161098
1,-1.240781,-1.401992,-0.453874
2,0.063155,-1.401992,-1.314886
3,-1.365589,-1.401992,0.531838
4,0.398513,NaN,0.531838
5,-0.345927,NaN,0.531838


In [172]:
data = Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [173]:
data.fillna(data.mean()) # 用统计量填充缺失值

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 层次化索引

### Series

In [174]:
data = Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                     [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]]) # index是列表的列表，按照外层到内层的顺序，每层列表长度一样
data

a  1    0.504193
   2   -0.268125
   3   -1.332757
b  1   -0.530113
   2    0.117977
   3    1.409926
c  1    0.448792
   2   -0.539606
d  2   -0.093771
   3    0.044860
dtype: float64

In [175]:
data.index # 返回MultiIndex对象

MultiIndex(levels=[[u'a', u'b', u'c', u'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

In [176]:
data['b':'c'] # 在外层索引中选取

b  1   -0.530113
   2    0.117977
   3    1.409926
c  1    0.448792
   2   -0.539606
dtype: float64

In [177]:
data[:, 2] # 在内层索引中选取

a   -0.268125
b    0.117977
c   -0.539606
d   -0.093771
dtype: float64

In [178]:
data.unstack() # 将Series按照层次化索引转化为DataFrame（数据透视表操作）

,1,2,3
a,0.504193,-0.268125,-1.332757
b,-0.530113,0.117977,1.409926
c,0.448792,-0.539606,NaN
d,NaN,-0.093771,0.044860


In [179]:
data.unstack().stack() # unstack的逆运算是stack

a  1    0.504193
   2   -0.268125
   3   -1.332757
b  1   -0.530113
   2    0.117977
   3    1.409926
c  1    0.448792
   2   -0.539606
d  2   -0.093771
   3    0.044860
dtype: float64

### DataFrame
DataFrame的每条轴都可以有分层索引

In [180]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame

Ohio 
 Colorado 
 
 
 
 
 Green 
 Red 
 Green 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

In [181]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color'] # 每层索引都可以有name属性
frame

state 
 Ohio 
 Colorado 
 
 
 
 color 
 Green 
 Red 
 Green 
 
 
 key1 
 key2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

In [182]:
frame['Ohio'] # 通过分层索引选取列分组

color 
 Green 
 Red 
 
 
 key1 
 key2 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 
 
 2 
 3 
 4 
 
 
 b 
 1 
 6 
 7 
 
 
 2 
 9 
 10

### 创建并使用MultiIndex

In [183]:
multiindex = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color']) # 创建一个层次化索引
frame.columns = multiindex # 将分层索引赋予某个DataFrame
frame

state 
 Ohio 
 Colorado 
 
 
 
 color 
 Green 
 Red 
 Green 
 
 
 key1 
 key2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

### 重排分级顺序

#### 分层索引互换
+ swaplevel方法，接受两个级别编号或名称，返回新对象

In [184]:
frame.swaplevel('key1', 'key2')

state 
 Ohio 
 Colorado 
 
 
 
 color 
 Green 
 Red 
 Green 
 
 
 key2 
 key1 
 
 
 
 
 
 
 
 1 
 a 
 0 
 1 
 2 
 
 
 2 
 a 
 3 
 4 
 5 
 
 
 1 
 b 
 6 
 7 
 8 
 
 
 2 
 b 
 9 
 10 
 11

In [185]:
frame.swaplevel(0, 1) # 以上两种方式等价，默认互换行标签

state 
 Ohio 
 Colorado 
 
 
 
 color 
 Green 
 Red 
 Green 
 
 
 key2 
 key1 
 
 
 
 
 
 
 
 1 
 a 
 0 
 1 
 2 
 
 
 2 
 a 
 3 
 4 
 5 
 
 
 1 
 b 
 6 
 7 
 8 
 
 
 2 
 b 
 9 
 10 
 11

In [186]:
frame.swaplevel(0, 1, axis= 1) # 互换列标签

color 
 Green 
 Red 
 Green 
 
 
 
 state 
 Ohio 
 Ohio 
 Colorado 
 
 
 key1 
 key2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

#### 分层索引排序
+ sortlevel方法，根据单个级别的索引排序

In [187]:
frame.sortlevel(1)

state 
 Ohio 
 Colorado 
 
 
 
 color 
 Green 
 Red 
 Green 
 
 
 key1 
 key2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 a 
 2 
 3 
 4 
 5 
 
 
 b 
 2 
 9 
 10 
 11

In [188]:
frame.swaplevel(0, 1).sortlevel(0) # swaplevel与sortlevel常常同时使用，使索引互换的结果以有序的形式返回

state 
 Ohio 
 Colorado 
 
 
 
 color 
 Green 
 Red 
 Green 
 
 
 key2 
 key1 
 
 
 
 
 
 
 
 1 
 a 
 0 
 1 
 2 
 
 
 b 
 6 
 7 
 8 
 
 
 2 
 a 
 3 
 4 
 5 
 
 
 b 
 9 
 10 
 11

### 根据级别汇总统计
许多pandas的描述和汇总统计都有一个level选项，用于指定在某条轴上汇总统计的级别

In [189]:
frame.sum(level='key2')

state 
 Ohio 
 Colorado 
 
 
 color 
 Green 
 Red 
 Green 
 
 
 key2 
 
 
 
 
 
 
 
 1 
 6 
 8 
 10 
 
 
 2 
 12 
 14 
 16

In [190]:
frame.sum(level='color', axis=1)

color 
 Green 
 Red 
 
 
 key1 
 key2 
 
 
 
 
 
 
 a 
 1 
 2 
 1 
 
 
 2 
 8 
 4 
 
 
 b 
 1 
 14 
 7 
 
 
 2 
 20 
 10

### 使用DataFrame的列

In [191]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


#### 列转换为行索引

In [192]:
frame2 = frame.set_index(['c', 'd']) # 列名自动转换为行标签的name属性
frame2

a 
 b 
 
 
 c 
 d 
 
 
 
 
 
 
 one 
 0 
 0 
 7 
 
 
 1 
 1 
 6 
 
 
 2 
 2 
 5 
 
 
 two 
 0 
 3 
 4 
 
 
 1 
 4 
 3 
 
 
 2 
 5 
 2 
 
 
 3 
 6 
 1

In [193]:
frame.set_index(['c', 'd'], drop=False) # 默认列从DataFrame中移除，设置drop参数保留列

a 
 b 
 c 
 d 
 
 
 c 
 d 
 
 
 
 
 
 
 
 
 one 
 0 
 0 
 7 
 one 
 0 
 
 
 1 
 1 
 6 
 one 
 1 
 
 
 2 
 2 
 5 
 one 
 2 
 
 
 two 
 0 
 3 
 4 
 two 
 0 
 
 
 1 
 4 
 3 
 two 
 1 
 
 
 2 
 5 
 2 
 two 
 2 
 
 
 3 
 6 
 1 
 two 
 3

####  行索引转换为列

In [194]:
frame2.reset_index() # 可以设置转换的行标签的编号、名称，name属性转换为列名

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 其他有关pandas的话题

### 整数索引

#### 在pandas中尽量避免使用整数索引，而使用标签索引

+ 非数值标签

In [195]:
ser = Series(np.arange(3.), index=['a', 'b', 'c'])
ser

a    0.0
b    1.0
c    2.0
dtype: float64

In [196]:
ser[1] # 当标签非数值时，整数索引基于位置

1.0

In [197]:
ser[-1] # 当标签非数值时，整数索引基于位置

2.0

+ 数值标签

In [198]:
ser2 = Series(np.arange(3.), index=[1, 2, 3])
ser2

1    0.0
2    1.0
3    2.0
dtype: float64

In [199]:
ser2[1] # 当标签为数值时，整数索引基于标签

0.0

In [200]:
ser2.ix[:2] # 轴索引含有索引器，基于标签

1    0.0
2    1.0
dtype: float64

In [201]:
ser2.iloc[-1] # 解决办法，用.iloc方法，基于位置索引

2.0

#### 获取可靠的、不考虑索引类型的、基于位置的索引
+ iloc方法

In [202]:
ser3 = Series(range(3), index=[-5, 1, 3])
ser3

-5    0
 1    1
 3    2
dtype: int64

In [203]:
ser3.iloc[2]

2

In [204]:
frame = DataFrame(np.arange(6).reshape((3, 2)), index=[2, 0, 1])
frame

,0,1
2,0,1
0,2,3
1,4,5


In [205]:
frame.iloc[0] # 按行索引

0    0
1    1
Name: 2, dtype: int32

### 面板数据
Panel数据结构可以被看做三维版的DataFrame。层次化索引使得多数情况下没有必要使用N维数组

In [206]:
import pandas_datareader.data as web

pdata = pd.Panel(dict((stk, web.get_data_yahoo(stk))
                       for stk in ['AAPL', 'GOOG', 'MSFT', 'DELL']))
pdata

<class 'pandas.core.panel.Panel'>
Dimensions: 4 (items) x 1727 (major_axis) x 6 (minor_axis)
Items axis: AAPL to MSFT
Major_axis axis: 2010-01-04 00:00:00 to 2016-10-12 00:00:00
Minor_axis axis: Open to Adj Close

In [207]:
pdata = pdata.swapaxes('items', 'minor')
pdata['Adj Close']

,AAPL,DELL,GOOG,MSFT
Date,,,,
2010-01-04,27.990226,14.06528,313.062468,25.884104
2010-01-05,28.038618,14.38450,311.683844,25.892466
2010-01-06,27.592626,14.10397,303.826685,25.733566
2010-01-07,27.541619,14.23940,296.753749,25.465944
2010-01-08,27.724725,14.36516,300.709808,25.641571
2010-01-11,27.480148,14.37483,300.255255,25.315406
2010-01-12,27.167562,14.56830,294.945572,25.148142
2010-01-13,27.550775,14.57797,293.252243,25.382312
2010-01-14,27.391211,14.22005,294.630868,25.892466


#### 面板数据的索引
+ ix方法，索引的顺序为items（列标签）、major（时序）、minor（行标签）

In [208]:
pdata.ix[:, '6/1/2012', :]

,Open,High,Low,Close,Volume,Adj Close
AAPL,569.159996,572.650009,560.520012,560.989983,130246900.0,73.371509
DELL,12.150000,12.300000,12.045000,12.070000,19397600.0,11.675920
GOOG,571.790972,572.650996,568.350996,570.981000,6138700.0,285.205295
MSFT,28.760000,28.959999,28.440001,28.450001,56634300.0,25.262972


In [209]:
pdata.ix['Adj Close', '5/22/2012':, :]

,AAPL,DELL,GOOG,MSFT
Date,,,,
2012-05-22,72.845742,14.58765,300.100412,26.426222
2012-05-23,74.623162,12.08221,304.426106,25.849037
2012-05-24,73.937831,12.04351,301.528978,25.813517
2012-05-25,73.541536,12.05319,295.470050,25.804637
2012-05-28,NaN,12.05319,NaN,NaN
2012-05-29,74.846812,12.24666,296.873645,26.248626
2012-05-30,75.749262,12.14992,293.821674,26.053272
2012-05-31,75.560928,11.92743,290.140354,25.920075
2012-06-01,73.371509,11.67592,285.205295,25.262972


#### 面板数据与“堆积式”DataFrame（层次化索引）的转换

In [210]:
stacked = pdata.ix[:, '5/30/2012':, :].to_frame() # 面板数据转化为DataFrame
stacked

Open 
 High 
 Low 
 Close 
 Volume 
 Adj Close 
 
 
 Date 
 minor 
 
 
 
 
 
 
 
 
 
 
 2012-05-30 
 AAPL 
 569.199997 
 579.989990 
 566.559990 
 579.169998 
 132357400.0 
 75.749262 
 
 
 DELL 
 12.590000 
 12.700000 
 12.460000 
 12.560000 
 19787800.0 
 12.149920 
 
 
 GOOG 
 588.161028 
 591.901014 
 583.530999 
 588.230992 
 3827600.0 
 293.821674 
 
 
 MSFT 
 29.350000 
 29.480000 
 29.120001 
 29.340000 
 41585500.0 
 26.053272 
 
 
 2012-05-31 
 AAPL 
 580.740021 
 581.499985 
 571.460022 
 577.730019 
 122918600.0 
 75.560928 
 
 
 DELL 
 12.530000 
 12.540000 
 12.330000 
 12.330000 
 19955600.0 
 11.927430 
 
 
 GOOG 
 588.720982 
 590.001032 
 579.001013 
 580.860990 
 5958800.0 
 290.140354 
 
 
 MSFT 
 29.299999 
 29.420000 
 28.940001 
 29.190001 
 39134000.0 
 25.920075 
 
 
 2012-06-01 
 AAPL 
 569.159996 
 572.650009 
 560.520012 
 560.989983 
 130246900.0 
 73.371509 
 
 
 DELL 
 12.150000 
 12.300000 
 12.045000 
 12.070000 
 19397600.0 
 11.675920 
 
 
 GOOG 
 571.790972 
 572.650996 
 568.350996 
 570.981000 
 6138700.0 
 285.205295 
 
 
 MSFT 
 28.760000 
 28.959999 
 28.440001 
 28.450001 
 56634300.0 
 25.262972 
 
 
 2012-06-04 
 AAPL 
 561.500008 
 567.499985 
 548.499977 
 564.289978 
 139248900.0 
 73.803114 
 
 
 DELL 
 12.110000 
 12.112500 
 11.800000 
 12.000000 
 17015700.0 
 11.608210 
 
 
 GOOG 
 570.220958 
 580.491016 
 570.011006 
 578.590973 
 4883500.0 
 289.006480 
 
 
 MSFT 
 28.620001 
 28.780001 
 28.320000 
 28.549999 
 47926300.0 
 25.351768 
 
 
 2012-06-05 
 AAPL 
 561.269989 
 566.470001 
 558.330002 
 562.830025 
 97053600.0 
 73.612167 
 
 
 DELL 
 11.950000 
 12.240000 
 11.950000 
 12.160000 
 15620900.0 
 11.762980 
 
 
 GOOG 
 575.451008 
 578.131003 
 566.470986 
 570.410999 
 4697200.0 
 284.920579 
 
 
 MSFT 
 28.510000 
 28.750000 
 28.389999 
 28.510000 
 45715400.0 
 25.316250 
 
 
 2012-06-06 
 AAPL 
 567.770004 
 573.849983 
 565.499992 
 571.460022 
 100363900.0 
 74.740879 
 
 
 DELL 
 12.210000 
 12.280000 
 12.090000 
 12.215000 
 20779900.0 
 11.816190 
 
 
 GOOG 
 576.480979 
 581.970971 
 573.611004 
 580.570966 
 4207200.0 
 289.995487 
 
 
 MSFT 
 28.879999 
 29.370001 
 28.809999 
 29.350000 
 46860500.0 
 26.062152 
 
 
 2012-06-07 
 AAPL 
 577.290009 
 577.320023 
 570.500000 
 571.720001 
 94941700.0 
 74.774881 
 
 
 DELL 
 12.320000 
 12.410000 
 12.120000 
 12.130000 
 20074000.0 
 11.733960 
 
 
 GOOG 
 587.601014 
 587.891038 
 577.251006 
 578.230986 
 3530100.0 
 288.826666 
 
 
 MSFT 
 29.639999 
 29.700001 
 29.170000 
 29.230000 
 37792800.0 
 25.955594 
 
 
 2012-06-08 
 AAPL 
 571.599998 
 580.580017 
 568.999992 
 580.319984 
 86879100.0 
 75.899667 
 
 
 DELL 
 12.130000 
 12.225000 
 12.020000 
 12.120000 
 18155600.0 
 11.724290 
 
 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 
 
 2016-09-29 
 AAPL 
 113.160004 
 113.800003 
 111.800003 
 112.180000 
 35887000.0 
 112.180000 
 
 
 GOOG 
 781.440002 
 785.799988 
 774.231995 
 775.010010 
 1301400.0 
 775.010010 
 
 
 MSFT 
 57.810001 
 58.169998 
 57.209999 
 57.400002 
 25463500.0 
 57.400002 
 
 
 2016-09-30 
 AAPL 
 112.459999 
 113.370003 
 111.800003 
 113.050003 
 36379100.0 
 113.050003 
 
 
 GOOG 
 776.330017 
 780.940002 
 774.090027 
 777.289978 
 1580300.0 
 777.289978 
 
 
 MSFT 
 57.570000 
 57.770000 
 57.340000 
 57.599998 
 29910800.0 
 57.599998 
 
 
 2016-10-03 
 AAPL 
 112.709999 
 113.050003 
 112.279999 
 112.519997 
 21701800.0 
 112.519997 
 
 
 GOOG 
 774.250000 
 776.065002 
 769.500000 
 772.559998 
 1275900.0 
 772.559998 
 
 
 MSFT 
 57.410000 
 57.549999 
 57.060001 
 57.419998 
 19189500.0 
 57.419998 
 
 
 2016-10-04 
 AAPL 
 113.059998 
 114.309998 
 112.629997 
 113.000000 
 29736800.0 
 113.000000 
 
 
 GOOG 
 776.030029 
 778.710022 
 772.890015 
 776.429993 
 1197400.0 
 776.429993 
 
 
 MSFT 
 57.270000 
 57.599998 
 56.970001 
 57.240002 
 20085900.0 
 57.240002 
 
 
 2016-10-05 
 AAPL 
 113.400002 
 113.660004 
 112.690002 
 113.050003 
 214

In [211]:
stacked.to_panel() # DataFrame转化为面板数据

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 1114 (major_axis) x 4 (minor_axis)
Items axis: Open to Adj Close
Major_axis axis: 2012-05-30 00:00:00 to 2016-10-12 00:00:00
Minor_axis axis: AAPL to MSFT